# Purpose

Figure out pulling location IDs for only DCFC US (non-Tesla to start with) in the US. Need location IDs to efficiently scrape Plugshare with some of our other code.

# Imports

In [1]:

%load_ext autoreload
%autoreload 2

import numpy as np
from rich import print
import os
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from typing import List, Union, Set

# from torch.utils.data import DataLoader

from evlens.data.plugshare import MainMapScraper

# Electrify America in Springfield, VA mall parking lot
TEST_LOCATION = 252784

from dotenv import load_dotenv
load_dotenv(override=True)

from evlens.logs import setup_logger
logger = setup_logger("Notebook-0.3")
logger.info("TEST!")

2024-07-02_T07_32_40CDT: INFO (Notebook-0.3:L25) - TEST!


# Set up our constants

In [2]:
# URL = "https://developer.plugshare.com/embed"
location_id = 252784
URL = f"https://www.plugshare.com/location/{location_id}"

# Try uploading a screenshot to GCP Cloud Storage

This isn't datatabase stuff but useful for scraping and *should* be easy to do...

In [7]:
from evlens.data.google_cloud import upload_file

upload_file(
    bucket_name='plugshare_scraping',
    source_filepath='../../data/external/plugshare/06-30-2024/errors/07-01-2024_T04_37_46_checkins.png', 
    destination_blob_name='errors/07-01-2024_T04_37_46_checkins.png'
)

2024-07-01_T17_53_05CDT: INFO (evlens.data.google_cloud:L41) - File ../../data/external/plugshare/06-30-2024/errors/07-01-2024_T04_37_46_checkins.png uploaded to errors/07-01-2024_T04_37_46_checkins.png.


In [5]:
# Push our hex data for location ID scraping to the bucket
from evlens.data.google_cloud import upload_file

source_path = "/Users/davemcrench/Projects/evlens/data/processed/h3_hexagons_usa.pkl"

upload_file(
    bucket_name='plugshare_scraping',
    source_filepath=source_path,
    destination_blob_name='location_id_scraping/h3_hexagons_usa.pkl'
)

ConnectionError: ('Connection aborted.', TimeoutError('The write operation timed out'))

# Working with Google Cloud BigQuery

It has a free tier and seems to be a good choice for our needs at the moment, at least for an MVP.

In [ ]:
# Test grabbing some data
from google.cloud import bigquery

client = bigquery.Client()

# Perform a query.
QUERY = (
    'SELECT name FROM `bigquery-public-data.usa_names.usa_1910_2013` '
    'WHERE state = "TX" '
    'LIMIT 100')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row.name)

In [4]:
from evlens.data.google_cloud import bigquery_to_dataframe

query = """
SELECT name, SUM(number) as count
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY name
ORDER BY count DESC
LIMIT 10
"""

df = bigquery_to_dataframe(query)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    10 non-null     object
 1   count   10 non-null     Int64 
dtypes: Int64(1), object(1)
memory usage: 302.0+ bytes


,name,count
0,James,5054074
1,John,4910976
2,Robert,4763102
3,Michael,4396482
4,William,3939912
